In [1]:
import d2lzh as d2l
import math
from mxnet import autograd, nd
from mxnet.gluon import loss as gloss
import time

In [2]:
(corpus_indices, char_to_idx, idx_to_char,
vocab_size) = d2l.load_data_jay_lyrics()

In [3]:
#one-hot
nd.one_hot(nd.array([0, 2]), vocab_size)


[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]]
<NDArray 2x1027 @cpu(0)>

In [4]:
def to_onehot(X, size):
    return [nd.one_hot(x, size) for x in X.T]

In [5]:
X = nd.arange(10).reshape((2, 5))
inputs = to_onehot(X, vocab_size)
len(inputs), inputs[0].shape

(5, (2, 1027))

In [6]:
#init parameters
num_inputs, num_hiddens, num_outputs = vocab_size, 256, vocab_size
ctx = d2l.try_gpu()
print('will use', ctx)

def get_params():
    def _one(shape):
        return nd.random.normal(scale=0.01, shape=shape, ctx=ctx)
    #hidden layer
    W_xh = _one((num_inputs, num_hiddens))
    W_hh = _one((num_hiddens, num_hiddens))
    b_h = nd.zeros(num_hiddens, ctx=ctx)
    #output layer
    W_hq = _one((num_hiddens, num_outputs))
    b_q = nd.zeros(num_outputs, ctx=ctx)
    #attach grad
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.attach_grad()
    return params

will use gpu(0)


In [7]:
#define model
def init_rnn_state(batch_size, num_hiddens, ctx):
    return (nd.zeros(shape=(batch_size, num_hiddens), ctx=ctx),)

In [8]:
def rnn(inputs, state, params):
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    for X in inputs:
        H = nd.tanh(nd.dot(X, W_xh) + nd.dot(H, W_hh) + b_h)
        Y = nd.dot(H, W_hq) + b_q
        outputs.append(Y)
    return outputs, (H,)

In [9]:
state = init_rnn_state(X.shape[0], num_hiddens, ctx)
inputs = to_onehot(X.as_in_context(ctx), vocab_size)
params = get_params()
outputs, state_new = rnn(inputs, state, params)
len(outputs), outputs[0].shape, state_new[0].shape

(5, (2, 1027), (2, 256))

In [10]:
#define predict function
def predict_rnn(prefix, num_chars, rnn, params, init_rnn_state,
               num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx):
    state = init_rnn_state(1, num_hiddens, ctx)
    output = [char_to_idx[prefix[0]]]
    for t in range(num_chars + len(prefix) - 1):
        #use last output as input
        X = to_onehot(nd.array([output[-1]], ctx=ctx), vocab_size)
        #get output and update hidden state
        (Y, state) = rnn(X, state, params)
        #
        if t < len(prefix) -1:
            output.append(char_to_idx[prefix[t + 1]])
        else:
            output.append(int(Y[0].argmax(axis=1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])  

In [11]:
predict_rnn('分开', 10, rnn, params, init_rnn_state, num_hiddens, vocab_size,
            ctx, idx_to_char, char_to_idx)

'分开滩强啊腐壁力闪乖耿千'

In [12]:
#clip gradient
def grad_clipping(params, theta, ctx):
    norm = nd.array([0], ctx)
    for param in params:
        norm += (param.grad ** 2).sum()
    norm = norm.sqrt().asscalar()
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [13]:
def train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                         vocab_size, ctx, corpus_indices, idx_to_char,
                         char_to_idx, is_random_iter, num_epochs, num_steps,
                         lr, clipping_theta, batch_size, pred_period,
                         pred_len, prefixes):
    if is_random_iter:
        data_iter_fn = d2l.data_iter_random
    else:
        data_iter_fn = d2l.data_iter_consecutive
    params = get_params()
    loss = gloss.SoftmaxCrossEntropyLoss()
    
    for epoch in range(num_epochs):
        if not is_random_iter:
            state = init_rnn_state(batch_size, num_hiddens, ctx)
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = data_iter_fn(corpus_indices, batch_size, num_steps, ctx)
        for X, Y in data_iter:
            if is_random_iter:
                state = init_rnn_state(batch_size, num_hiddens, ctx)
            else:
                for s in state:
                    s.detach()
            with autograd.record():
                inputs = to_onehot(X, vocab_size)
                (outputs, state) = rnn(inputs, state, params)
                outputs = nd.concat(*outputs, dim=0)
                y = Y.T.reshape((-1,))
                l = loss(outputs, y).mean()
            l.backward()
            grad_clipping(params, clipping_theta, ctx)
            d2l.sgd(params, lr, 1)
            l_sum += l.asscalar() * y.size
            n += y.size
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
            epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn(
                prefix, pred_len, rnn, params, init_rnn_state,
                num_hiddens, vocab_size, ctx, idx_to_char, char_to_idx))

In [14]:
num_epochs, num_steps, batch_size, lr, clipping_theta = 250, 35, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 50, 50, ['分开', '不分开']

In [15]:
train_and_predict_rnn(rnn, get_params, init_rnn_state, num_hiddens,
                      vocab_size, ctx, corpus_indices, idx_to_char,
                      char_to_idx, True, num_epochs, num_steps, lr,
                      clipping_theta, batch_size, pred_period, pred_len,
                      prefixes)

epoch 50, perplexity 69.736887, time 0.19 sec
 - 分开 我不要这 你有我有别你 一知就觉 我有你的可爱女人 坏明的让我疯狂的可爱女人 坏明的让我疯狂的可爱
 - 不分开 快颗用 一颗我 三子我 一子我 三子我 一子我 三子我 一子我 三子我 一子我 三子我 一子我 三
epoch 100, perplexity 9.939525, time 0.19 sec
 - 分开 一只用双截  爱悔着对我 相不寄以 我不要好生 我不能再想 我不 我不 我不要 爱情走的太快就像龙
 - 不分开吗 我不要再想 我不 我不 我不要 爱情走的太快就像龙卷风 不能承的我如 你却啊 你怎么 我想就这样
epoch 150, perplexity 2.853640, time 0.18 sec
 - 分开 一只在人落 谁话都停 在我心外 我跟再这现 让色蜡烛 温暖了空屋 白色蜡烛 温暖了空屋 白色蜡烛 
 - 不分开简 我不能再想 我不 我不 我不要再想你 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 
epoch 200, perplexity 1.563910, time 0.19 sec
 - 分开 一只令 心拳的母日语言等力 我们无你 你场盯人 不有再重 没有不重 你去操发 恨再不动 没有梦 痛
 - 不分开期简的胖女巫 用拉丁文念咒语啦啦呜 她养的黑猫笑起来像哭 啦啦啦呜 在小村外的溪边河口默默等著我 娘
epoch 250, perplexity 1.323317, time 0.19 sec
 - 分开 爱蟑在人教 到窝拽怕 全家怕空  伤种碰 你够了的烟味 漫哼完过死药是天肉 草原上两只敌对野牛 像
 - 不分开简简单单没有伤害 你 靠着你的肩膀 你 在我胸口睡著 像这样的生活 我爱你 你爱我 开不了口 周杰伦
